In [20]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hnmchallenge.dataset import Dataset
from hnmchallenge.filtered_dataset import FilterdDataset
from hnmchallenge.models.top_pop import TopPop
from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.constant import *
from hnmchallenge.models.sgmc.sgmc import SGMC
from hnmchallenge.models.ease.ease import EASE
from hnmchallenge.models.itemknn.itemknn import ItemKNN 

In [21]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

In [22]:
dataset = FilterdDataset()
dr = DataReader()


In [4]:
recom = SGMC(dataset, k=128, time_weight=True)

In [23]:
recom = ItemKNN(dataset, topk=1000, time_weight=True)

In [4]:
recom = EASE(dataset, l2=0.00000001, time_weight=True)

In [24]:
train = dataset.get_train_df()
train_sub = dataset.get_train_df_user_subset()
fd = dr.get_filtered_full_data()

In [25]:
recom.compute_similarity_matrix(train)  

INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 1128716


func:interactions_to_sparse_matrix
 took: 8.841273307800293 sec


In [ ]:
recs = recom.recommend(interactions=train_sub, batch_size=10_000, remove_seen=False)

In [26]:
recs = recom.recommend_multicore(interactions=train_sub, batch_size=10_000, num_cpus=20,
                                 remove_seen=False)

INFO:hnmchallenge.recommender_interface:Recommending items MULTICORE


  0%|          | 0/5 [00:00<?, ?it/s]

Predicting using time_weight importance...


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 9003
INFO:hnmchallenge.utils.sparse_matrix:Applying time weight on user-item interactions


func:interactions_to_sparse_matrix
 took: 0.44321417808532715 sec


...

INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...


Predicting using time_weight importance...


INFO:hnmchallenge.utils.pandas_utils:unique customer_id: 9002
INFO:hnmchallenge.utils.sparse_matrix:Applying time weight on user-item interactions


func:interactions_to_sparse_matrix
 took: 0.1939706802368164 sec


INFO:hnmchallenge.recommender_interface:SPARSE Item Similarity MUL...
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True
INFO:hnmchallenge.utils.sparse_matrix:Sort_top_k:True


In [27]:
map_at_k(rating_true=dataset.get_val_df(), rating_pred=recs)

0.02960409727623696

In [28]:
map_at_k(rating_true=dataset.get_test_df(), rating_pred=recs)

0.02988199549883098

In [9]:
from hnmchallenge.submission_handler import SubmissionHandler
sh = SubmissionHandler()                                               

In [10]:
sh.create_submission_filtered_data(recs, sub_name="ItemKNN-td-50")

INFO:hnmchallenge.submission_handler:Submission with Filtered Data: ItemKNN-td-50 created succesfully!


In [ ]:
user_map_dict, item_map_dict = dr.get_new_raw_mapping_dict()

In [ ]:
grp_recs_df = recs.groupby(DEFAULT_USER_COL)[DEFAULT_ITEM_COL].apply(list)
grp_recs_df = grp_recs_df.to_frame().reset_index()
# map back to original ids
grp_recs_df[DEFAULT_USER_COL] = grp_recs_df[DEFAULT_USER_COL].apply(
    lambda x: user_map_dict.get(x)
)

In [ ]:
grp_recs_df[DEFAULT_ITEM_COL] = grp_recs_df[DEFAULT_ITEM_COL].apply(
    lambda x: " ".join(list(map(item_map_dict.get, x)))
)

In [ ]:
grp_recs_df

In [ ]:
map_at_k(rating_true=test_df_s, rating_pred=recs)

In [ ]:
map_at_k(rating_true=val_df_s, rating_pred=recs)

In [ ]:
dr = DataReader()

In [ ]:
dr.get_zero_interactions_recs()

In [ ]:
zero_int_users